In [10]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_vit
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import data_generator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
   tf.config.set_visible_devices(gpus[0], 'GPU')
   tf.config.experimental.set_memory_growth(gpus[0], True)

In [11]:
model = lstm_vit()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 15, 256, 256, 3   0         
                             )]                                  
                                                                 
 time_distributed_7 (TimeDi  (None, 15, 32, 32, 256)   2487424   
 stributed)                                                      
                                                                 
 reshape_6 (Reshape)         (None, 15, 262144)        0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 15, 128)           134251008 
 onal)                                                           
                                                                 
 time_distributed_8 (TimeDi  (None, 15, 1024)          132096    
 stributed)                                                

In [12]:
train_gen = data_generator(
    image_dir="../ViT/data_v3_processed/train/images/images",
    mask_dir="../ViT/data_v3_processed/train/masks/masks",
    batch_size=64
)

In [13]:
val_gen = data_generator(
    image_dir="data_v3_processed/val/images/images",
    mask_dir="data_v3_processed/val/masks/masks",
    batch_size=64
)

X_batch, y_batch = next(train_gen)
print(f"Image batch shape: {X_batch.shape}")  # Should be (4, 256, 256, 3)
print(f"Mask batch shape: {y_batch.shape}")   # Should be (4, 256, 256, 1)
print(f"Image range: [{X_batch.min():.3f}, {X_batch.max():.3f}]")
print(f"Mask range: [{y_batch.min():.3f}, {y_batch.max():.3f}]")

Image batch shape: (196, 15, 256, 256, 3)
Mask batch shape: (196, 15, 256, 256, 1)
Image range: [0.000, 1.000]
Mask range: [0.000, 1.000]


In [14]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=1,
                    batch_size=64,
                    steps_per_epoch=1000,
                    validation_steps=13)

2025-08-03 13:33:51.334443: W external/local_tsl/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2025-08-03 13:33:51.727988: W tensorflow/core/kernels/gpu_utils.cc:54] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2025-08-03 13:33:51.754356: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-08-03 13:34:04.012812: W external/local_tsl/ts

ResourceExhaustedError: Graph execution error:

Detected at node model_3/time_distributed_7/model_2/conv2d_5/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/usr/local/

In [ ]:
model.save("lstm_vit_model.h5")